### 3-1. 문장의 어조(tone_s)를 구한 후 문서 전체의 어조(tone_i)를 구한다.

In [1]:
from ekonlpy.tag import Mecab
from ekonlpy.sentiment import MPCK
import pandas as pd
import numpy as np
mpck = MPCK()

In [2]:
# 단어 사전을 불러온다.
pos_dic = pd.read_csv('../datasets/3_pos_dict.csv')
neg_dic = pd.read_csv('../datasets/3_neg_dict.csv')

In [3]:
# 금통위 의사록을 불러온다.
df_3 = pd.read_csv('../datasets/minutes_with_label.csv')
del df_3['Unnamed: 0']
df_3 = df_3.dropna(axis=0)
df_3.tail()

,date,title,content,ngram,label
241,2020-01-17,제1차_금통위 의사록,일부 위원은 미국 ISM제조업지수의 최근 월 발표내용을 보면 다수의 산업에 서 수...,"['ism/NNG', '제조업/NNG', '지수/NNG', '발표/NNG', '내용...",same
242,2020-02-27,제4차_금통위 의사록,일부 위원은 국내경제가 올해 회복세로 전환될 것이라는 당초 전망의 중요한 전제 중...,"['경제/NNG', '회복/NNG', '전환/NNG', '되/VV', '전망/NNG...",down
246,2020-04-09,제8차_금통위 의사록,일부 위원은 관련부서에서 전망경로의 상하방 위험을 대체로 균형있게 고려 하고 있는...,"['전망/NNG', '경로/NNG', '상하방/NNG', '위험/NNG', '대체로...",down
249,2020-05-28,제12차_금통위 의사록,일부 위원은 코로나19 사태의 전개양상을 둘러싼 불확실성이 매우 높은 상황 에서 ...,"['코로나/NNG', '사태/NNG', '전개/NNG', '양상/NNG', '둘러싸...",down
252,2020-07-16,제15차_금통위 의사록,일부 위원은 지난 경제전망 이후 민간소비의 회복 속도가 예상보다 더딘 데 다 수출...,"['경제/NNG', '전망/NNG', '민간/NNG', '소비/NNG', '회복/N...",up


In [4]:
# 금통위 의사록 중 아무 데이터나 하나 고른다.
contents = df_3._get_value(235, 'content') 
contents = contents.replace('  ','').split('.')

In [5]:
# 톤을 구한다.
def tone(haw, dov):
    # tone 공식 : (No.of Hawkish - No.of Dovish) / (No.of Hawkish + No.of Dovish)
    bunja =  haw - dov
    bunmo =  haw + dov
    try:
        tone = bunja / bunmo
    except:
        tone=0
    
    return tone 

In [6]:
# 문장 내의 haw와 dov의 개수를 센다.
def cnt_haw_dov(ngram):
    haw, dov = 0, 0
    
    for n in ngram:
        if n in haw_ls:
            haw+=1
        elif n in dov_ls:
            dov+=1
            
    tone_s = tone(haw, dov)

    return np.where(tone_s>0, 'H', 'D')

In [7]:
haw_ls = pos_dic['ngram'].values.tolist()
dov_ls = neg_dic['ngram'].values.tolist()

Haw , Dov = 0, 0

for sents in contents:
    
    tokens = mpck.tokenize(sents)
    ngrams = mpck.ngramize(tokens)

    ngram = tokens+ngrams

    label = cnt_haw_dov(ngram)
    
    if label == 'H':
        Haw +=1
    else:
        Dov +=1
        
# 문서 전체의 톤을 구한다.
tone_i = tone(Haw, Dov)
print('tone_i = ',tone_i)

tone_i =  -0.7152317880794702
